# Init OpenSky into Kafka

In [25]:
import requests
from kafka import KafkaProducer
import json

correspond = ["icao24","callsign","origin_country","time_position",
              "last_contact","longitude","latitude","baro_altitude"
              ,"on_ground","velocity","true_track","vertical_rate",
              "sensors","geo_altitude","squawk","spi","position_source","category"]



def send_opensky_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from ThingSpeak
    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])
        state = []
        print(states)
        # Publish each entry to Kafka
        for field in fields:
            print(field)
            fieldIndex = correspond.index(field)
            print(fieldIndex)
            state.append({field:states[fieldIndex]})
        producer.send(topic, value=state)
        #print(f"Sent: {feed}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
        print(state)
    else:
        print(f"Failed to fetch data: {response.status_code}")

## Initiate beginning

In [22]:
from datetime import datetime, timedelta

# https://thingspeak.mathworks.com/channels/1785844 : Wind Power Smart Monitor
fields = {
    "on_ground": ["on_ground"],
    "latitude": ["latitude"],
    "longitude": ["longitude"]
}

api_url = "https://opensky-network.org/api/states/all"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

## Exemple with 10 calls.

In [26]:
import time 
start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

print(f"{start} - {end}")
send_opensky_to_kafka("on_ground", api_url, fields)

time.sleep(5)

# Update time variables
initial_date = end_date
end_date = initial_date + timedelta(minutes=step)

2025-01-25 17:00:00 - 2025-01-25 18:00:00
[['511142', 'ESTWN   ', 'Estonia', 1742294559, 1742294559, 8.7407, 46.0142, 624.84, False, 69.56, 137.7, -1.3, None, 784.86, None, False, 0], ['e49406', 'GLO1614 ', 'Brazil', 1742294840, 1742294840, -46.0189, -23.0549, 7757.16, False, 212.15, 12.89, 9.43, None, 8206.74, None, False, 0], ['738a88', '0000313 ', 'Israel', 1742294840, 1742294840, 34.727, 31.7615, 45.72, False, 0.51, 180, -1.63, None, 53.34, '0613', False, 0], ['4b1816', 'SWR5QD  ', 'Switzerland', 1742294840, 1742294840, 0.1553, 51.2975, 2667, False, 137.03, 253.2, -5.85, None, 2773.68, '3014', False, 0], ['4b1819', 'SWR5CP  ', 'Switzerland', 1742294840, 1742294841, 10.1995, 59.6051, 6454.14, False, 195.79, 26.7, -10.4, None, 6499.86, '3025', False, 0], ['e8027e', 'LPE2291 ', 'Chile', 1742294840, 1742294840, -77.7384, -9.8496, 10271.76, False, 231.29, 347.54, 6.18, None, 10972.8, '1777', False, 0], ['408120', 'VIR6J   ', 'United Kingdom', 1742294840, 1742294841, -0.2761, 51.2753, 26